# SageMaker を使った Keras Sequential モデルの学習
ここでは Keras を使ったサンプルコードを題材に、Amazon SageMaker への移行方法を順を追って説明します。このノートブックで用いるモデルは [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py) でも紹介さている CNN モデルになります。

## データセット
本ハンズオンでは [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) という機械学習では最も有名なデータセットの一つを使います。32✕32ピクセル、10個の異なるクラスからなる60,000枚の画像を分類します。下記にランダムに選んできた画像をご紹介します。

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

## データの準備
cifar10 の tfrecord 形式のデータセットを `s3://floor28/data/cifar10` からノートブックインスタンス上へ AWS CLI コマンドを使ってダウンロードしてきます。

In [1]:
!aws s3 cp --recursive s3://floor28/data/cifar10 ./data

download: s3://floor28/data/cifar10/eval/eval.tfrecords to data/eval/eval.tfrecords
download: s3://floor28/data/cifar10/validation/validation.tfrecords to data/validation/validation.tfrecords
download: s3://floor28/data/cifar10/train/train.tfrecords to data/train/train.tfrecords


## ローカルで書き換え前の学習スクリプトの実行

学習スクリプトは設定のために下記の引数が必要です。

1. model_dir : ログやチェックポイントを保存するためのパス
2. train, validation, eval : それぞれのtfrecordデータを保存するためのパス

学習スクリプトをノートブックインスタンスの環境で実行してみましょう。

In [ ]:
!mkdir -p logs
!python training_script/cifar10_keras.py --model_dir ./logs \
                                         --train data/train \
                                         --validation data/validation \
                                         --eval data/eval \
                                         --epochs 1
!rm -rf logs

## TensorFlow スクリプトモードでの学習
TensorFlow versions 1.11 以降では, Amazon SageMaker Python SDK ではスクリプトモードをサポートします。SageMaker で TensorFlow トレーニングスクリプトを最小限の変更で実行できます。SageMaker Python SDK は、 SageMaker トレーニングインスタンスへのスクリプトの転送を処理します。トレーニングインスタンスでは、SageMaker のネイティブ TensorFlow サポートがトレーニング関連の環境変数を設定し、トレーニングスクリプトを実行します。

スクリプトモードは Python 2.7- と Python 3.6- の両方でお使い頂けます。また、Horovod による分散学習にも対応してます。詳細は[コチラ](https://sagemaker.readthedocs.io/en/stable/using_tf.html)をご確認下さい。

### 学習スクリプトを SageMaker 向けに書き換える
SageMaker の学習インスタンスでは学習用のコンテナへ Amazon S3 に保存されたデータをダウンロードし学習へ活用します。その際、S3 バケットのデータのパスとコンテナ内のデータのパスを、コンテナの環境変数を介して関連付けます。また、学習によって作成された学習済モデルやそのチェックポイントなどの生成物も同様に環境変数と関連付けられます。

今回の cifer10 のデータセットでは Train、Validation、Eval の3種類のデータがあるため下記のように紐付けます。


|  S3 location  |  環境変数  |  値  |
| :---- | :---- | :----| 
|  s3://bucket_name/prefix/train  |  `SM_CHANNEL_TRAIN`  | `/opt/ml/input/data/train`  |
|  s3://bucket_name/prefix/validation  |  `SM_CHANNEL_VALIDATION`  | `/opt/ml/input/data/validation`  |
|  s3://bucket_name/prefix/eval  |  `SM_CHANNEL_EVAL`  | `/opt/ml/input/data/eval`  |
|  s3://bucket_name/prefix/model.tar.gz  |  `SM_MODEL_DIR`  |  `/opt/ml/model`  |
|  s3://bucket_name/prefix/output.tar.gz  |  `SM_OUTPUT_DATA_DIR`  |  `/opt/ml/output/data`  |

詳細は SageMaker Python SDK の[ドキュメント](https://sagemaker.readthedocs.io/en/stable/using_tf.html#preparing-a-script-mode-training-script)をご確認下さい。また [Amazon SageMaker で簡単に Keras を使う方法](https://aws.amazon.com/jp/blogs/news/amazon-sagemaker-keras/)というブログ記事もご参考に下さい。

#### **オリジナルの学習スクリプトである`training_script/cifar10_keras.py`をコピーした上で、`training_script/cifar10_keras_sm.py.`として保存して下さい。新しいファイルを書き換え用のファイルとして使います。**

このサンプルコードではネットワーク遅延をへらすために、モデルのチェックポイントをローカル環境へ保存します。これらは学習ジョブが終了した際に s3 へアップロードすることができます。

**①下記を `cifar10_keras_sm.py` の`if __name__ == ‘__main__’:` ブロックの中へ追加して下さい。**

```python
parser.add_argument(
        '--model_output_dir',
        type=str,
        default=os.environ.get('SM_MODEL_DIR'))
```


**② `default=os.environ['SM_CHANNEL_XXXX']` を train、validation、eval のそれぞれに追加して下さい。**

```python
parser.add_argument(
    '--train',
    type=str,
    required=False,
    default=os.environ['SM_CHANNEL_TRAIN'], 
    help='The directory where the CIFAR-10 input data is stored.')
    
parser.add_argument(
    '--validation',
    type=str,
    required=False,
    default=os.environ['SM_CHANNEL_VALIDATION'],
    help='The directory where the CIFAR-10 input data is stored.')
    
parser.add_argument(
    '--eval',
    type=str,
    required=False,
    default=os.environ['SM_CHANNEL_EVAL'],
    help='The directory where the CIFAR-10 input data is stored.')
```

**③`ModelCheckPoint` ラインを新しい保存先を使うよう変更します。**
```python
callbacks.append(ModelCheckpoint(args.model_output_dir + '/checkpoint-{epoch}.h5'))
```

**④`save_model call` を新しい保存先を呼び出すように変更します。**
  ```python
return save_model(model, args.model_dir)
```
から、
```python
return save_model(model, args.model_output_dir)
```
へ変更します。

### SageMaker ローカルモードを使った学習スクリプト書き換えの検証
トレーニングジョブを始める前に、ローカルモードを使って、このノートブックインスタンス上でコンテナを立てコードをデバッグしましょう。ローカルモードでは、Docker compose と NVIDIA Docker を使い、Amazon ECS から Amazon SageMaker TensorFlow コンテナをダウンロードしてきて使います。これにより、SageMaker Python SDK は CPU (single and multi-instance) や GPU (single instance) をエミュレートした環境をノートブックインスタンス上に構築します。

ローカルモードを使うことによって、コードの素早い検証を行ったり、既にお持ちの学習環境があればそのハードウェア資産を有効活用したりすることなどが可能になります。

In [2]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

`from sagemaker.tensorflow import TensorFlow` で読み込んだ SageMaker Python SDK の TensorFlow Estimator を作ります。詳細は[こちら](https://sagemaker.readthedocs.io/en/stable/using_tf.html#training-with-tensorflow-estimator)をご確認下さい。

In [3]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 20},
                       train_instance_count=1, train_instance_type='local')

今回使うデータセットに合わせて3つのチャネルとそのデータのパスを指定します。今回はローカルモードなので、ノートブックインスタンス上のパスを指定しています。

In [ ]:
estimator.fit({'train' :  'file://data/train',
               'validation' :  'file://data/validation',
               'eval' :  'file://data/eval'})

### Using SageMaker for faster training time
SageMakerでの学習にGPUインスタンスを使うことで学習時間を速めることが出来ます。学習を始める前に、予め Amazon S3 にデータを準備しておく必要があります。このノートブックを使ってその作業をします。

In [4]:
dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-us-east-1-815969174475/data/DEMO-cifar10'

今回は **ml.p3.2xlarge** インスタンスを使用し、エポック数を **epochs:20** と指定します。

In [5]:
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_sample0.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 1},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge')

学習ジョブを発行します。今回はそれぞれのチャネルに S3 のデータ保存先を指定します。

In [6]:
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2019-10-01 02:24:34 Starting - Starting the training job...
2019-10-01 02:24:35 Starting - Launching requested ML instances......
2019-10-01 02:25:36 Starting - Preparing the instances for training......
2019-10-01 02:26:53 Downloading - Downloading input data...
2019-10-01 02:27:21 Training - Downloading the training image...
2019-10-01 02:27:45 Training - Training image download completed. Training in progress.2019-10-01 02:27:48,940 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-10-01 02:27:49,421 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparamet


```
Billable seconds: <time>
```

と出力されればトレーニング終了です。これが実際にトレーニングインスタンスが課金される時間となります。